# Library Initilization

In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sns
import datetime as dt
from scipy import stats
import os
import glob
sns.set()
pd.options.mode.chained_assignment = None  # default='warn'
%matplotlib inline

## Cleaning functions

In [77]:
def item_header_clean(df):
    column_map = {
    "index" : "League",
    "League": "Date",
    "Date": "Id",
    "Id" : "Type"
    }
    df_indexreset = df.reset_index()
    df_renamed = df_indexreset.rename(columns = column_map)
    return df_renamed

def item_info_remove(df):
    items_to_drop = ["Prophecy", "DivinationCard"]
    cols_to_drop = ["Links", "Variant"]
    df_cols_dropped = df.drop(cols_to_drop, axis = 1)
    df_all_dropped = df_cols_dropped.loc[~df_cols_dropped["Type"].isin(items_to_drop)]
    return df_all_dropped

def item_top_15_filter(df):
    median_series = df.groupby("Name")["Value"].agg("median")
    sorted_series = median_series.sort_values(ascending = False)
    top_15_list = list(sorted_series[:15].index)
    df_top_15 = df.loc[df["Name"].isin(top_15_list)]
    return df_top_15

def add_relative_date(df):
    df_copy = df
    df_copy["Date"] = df_copy["Date"].astype('datetime64[D]')
    startdate = df_copy["Date"].min()
    df_copy["RelativeDate"] = (df_copy["Date"] - startdate)
    df_edited = df_copy.drop("Date", axis = 1)
    df_edited["RelativeDate"] = pd.to_timedelta(df_edited["RelativeDate"], unit = "D")
    return df_edited

def league_lifespan(row):
    early_league = dt.timedelta(days = 14)
    mid_league = dt.timedelta(days = 60)
    if row["RelativeDate"] <= early_league:
        return "Early"
    elif row["RelativeDate"] <= mid_league:
        return "Mid"
    return "End"

def item_file_clean(df):
    df1 = item_header_clean(df)
    df2 = item_info_remove(df1)
    df3 = item_top_15_filter(df2)
    df4 = add_relative_date(df3)
    df4["League Lifespan"] = df4.apply(league_lifespan, axis = 1)
    return df4

def currency_info_remove(df):
    currency_to_drop = ["Portal Scroll", "Scroll of Wisdom", "Armourer's Scrap", "Perandus Coin", "Orb of Transmutation", "Blacksmith's Whetstone", "Orb of Augmentation", "Orb of Alteration", "Splinter of Tul", "Chromatic Orb", "Splinter of Esh", "Splinter of Xoph", "Orb of Chance", "Glassblower's Bauble", "Splinter of Uul-Netol", "Silver Coin"]
    df_currency_dropped = df.loc[~(df["Get"].isin(currency_to_drop) | df["Pay"].isin(currency_to_drop))]
    return df_currency_dropped

def currency_file_clean(df):
    df1 = currency_info_remove(df)
    df2 = add_relative_date(df1)
    df2["League Lifespan"] = df2.apply(league_lifespan, axis = 1)
    return df2

## Data Import

In [78]:
breach_currency = pd.read_csv("1_currency.csv", sep = ";", index_col = None)
breach_items = pd.read_csv("1_items.csv", sep = ";", index_col = None)
breach_currency.head()

,League,Date,Get,Pay,Value,Confidence
0,Breach,2016-12-02,Silver Coin,Chaos Orb,0.96667,High
1,Breach,2016-12-03,Silver Coin,Chaos Orb,0.21937,High
2,Breach,2016-12-04,Silver Coin,Chaos Orb,0.18339,High
3,Breach,2016-12-05,Silver Coin,Chaos Orb,0.23571,High
4,Breach,2016-12-06,Silver Coin,Chaos Orb,0.26395,High


In [79]:
breach_items_indexreset = breach_items.reset_index()
breach_items_indexreset.head()

,index,League,Date,Id,Name,BaseType,Variant,Links,Value,Confidence
0,Breach,2016-12-02,1079,UniqueArmour,Sadima's Touch,Wool Gloves,NaN,1-4 links,0.78800,High
1,Breach,2016-12-03,1079,UniqueArmour,Sadima's Touch,Wool Gloves,NaN,1-4 links,0.29912,High
2,Breach,2016-12-04,1079,UniqueArmour,Sadima's Touch,Wool Gloves,NaN,1-4 links,0.15429,High
3,Breach,2016-12-05,1079,UniqueArmour,Sadima's Touch,Wool Gloves,NaN,1-4 links,0.18333,High
4,Breach,2016-12-06,1079,UniqueArmour,Sadima's Touch,Wool Gloves,NaN,1-4 links,0.20815,High


In [80]:
column_map = {
    "index" : "League",
    "League": "Date",
    "Date": "Id",
    "Id" : "Type"
}
breach_items = breach_items_indexreset.rename(columns = column_map)
breach_items.head()

,League,Date,Id,Type,Name,BaseType,Variant,Links,Value,Confidence
0,Breach,2016-12-02,1079,UniqueArmour,Sadima's Touch,Wool Gloves,NaN,1-4 links,0.78800,High
1,Breach,2016-12-03,1079,UniqueArmour,Sadima's Touch,Wool Gloves,NaN,1-4 links,0.29912,High
2,Breach,2016-12-04,1079,UniqueArmour,Sadima's Touch,Wool Gloves,NaN,1-4 links,0.15429,High
3,Breach,2016-12-05,1079,UniqueArmour,Sadima's Touch,Wool Gloves,NaN,1-4 links,0.18333,High
4,Breach,2016-12-06,1079,UniqueArmour,Sadima's Touch,Wool Gloves,NaN,1-4 links,0.20815,High


In [81]:
breach_currency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9932 entries, 0 to 9931
Data columns (total 6 columns):
League        9932 non-null object
Date          9932 non-null object
Get           9932 non-null object
Pay           9932 non-null object
Value         9932 non-null float64
Confidence    9932 non-null object
dtypes: float64(1), object(5)
memory usage: 465.6+ KB


# Wrangling the item dataset

In [82]:
breach_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133166 entries, 0 to 133165
Data columns (total 10 columns):
League        133166 non-null object
Date          133166 non-null object
Id            133166 non-null int64
Type          133166 non-null object
Name          133166 non-null object
BaseType      100461 non-null object
Variant       18075 non-null object
Links         54672 non-null object
Value         133166 non-null float64
Confidence    133166 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 10.2+ MB


In [83]:
breach_items.columns

Index(['League', 'Date', 'Id', 'Type', 'Name', 'BaseType', 'Variant', 'Links',
       'Value', 'Confidence'],
      dtype='object')

## Removing Unnecessary Information and Prophecies and Divination Cards

In [84]:
breach_items_edited1 = breach_items.drop(["Links", "Variant"], axis = 1)
breach_items_edited1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133166 entries, 0 to 133165
Data columns (total 8 columns):
League        133166 non-null object
Date          133166 non-null object
Id            133166 non-null int64
Type          133166 non-null object
Name          133166 non-null object
BaseType      100461 non-null object
Value         133166 non-null float64
Confidence    133166 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 8.1+ MB


In [85]:
_ = breach_items_edited1[breach_items_edited1.BaseType.isnull()]
_.Type.value_counts()

Prophecy          17317
DivinationCard    15388
Name: Type, dtype: int64

In [86]:
toDropList = ["Prophecy", "DivinationCard"]
breach_items_edited2 = breach_items_edited1.loc[~breach_items_edited1["Type"].isin(toDropList)]
breach_items_edited2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100461 entries, 0 to 133165
Data columns (total 8 columns):
League        100461 non-null object
Date          100461 non-null object
Id            100461 non-null int64
Type          100461 non-null object
Name          100461 non-null object
BaseType      100461 non-null object
Value         100461 non-null float64
Confidence    100461 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 6.9+ MB


In [87]:
medianseries = breach_items_edited2.groupby("Name")["Value"].agg("median")
sorted_series = medianseries.sort_values(ascending = False)
top_15 = list(sorted_series[:15].index)
top_15

['Headhunter',
 'Eyes of the Greatwolf',
 'Skyforth',
 "Rigwald's Savagery",
 'Natural Hierarchy',
 "Demigod's Bounty",
 "Atziri's Acuity",
 "Emperor's Mastery",
 "Rigwald's Command",
 "Atziri's Disfavour",
 "Demigod's Dominance",
 'The Red Nightmare',
 "Tukohama's Fortress",
 "Emperor's Wit",
 'Talisman of the Victor']

In [88]:
breach_items_top_15 = breach_items_edited2.loc[breach_items_edited2["Name"].isin(top_15)]
breach_items = breach_items_top_15
breach_items_top_15.head()

,League,Date,Id,Type,Name,BaseType,Value,Confidence
71448,Breach,2016-12-03,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,146.27273,Low
71449,Breach,2016-12-04,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,177.88472,Low
71450,Breach,2016-12-05,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,232.23558,Low
71451,Breach,2016-12-06,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,235.07981,Medium
71452,Breach,2016-12-07,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,313.95873,Medium


## Adding Relative Date Column

In [89]:
breach_items["Date"] = breach_items["Date"].astype('datetime64[D]')
breach_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1329 entries, 71448 to 132876
Data columns (total 8 columns):
League        1329 non-null object
Date          1329 non-null datetime64[ns]
Id            1329 non-null int64
Type          1329 non-null object
Name          1329 non-null object
BaseType      1329 non-null object
Value         1329 non-null float64
Confidence    1329 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 93.4+ KB


In [90]:
startdate = breach_items["Date"].min()
startdate

Timestamp('2016-12-03 00:00:00')

In [91]:
breach_items["RelativeDate"] = (breach_items["Date"] - startdate).astype('timedelta64[D]')

In [92]:
breach_edited = breach_items.drop("Date", axis = 1)

In [93]:
breach_edited["RelativeDate"] = pd.to_timedelta(breach_edited["RelativeDate"], unit = "D")

In [94]:
breach_edited.head()

,League,Id,Type,Name,BaseType,Value,Confidence,RelativeDate
71448,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,146.27273,Low,0 days
71449,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,177.88472,Low,1 days
71450,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,232.23558,Low,2 days
71451,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,235.07981,Medium,3 days
71452,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,313.95873,Medium,4 days


## Add column indicating league's lifespan

In [95]:
def league_lifespan(row):
    early_league = dt.timedelta(days = 14)
    mid_league = dt.timedelta(days = 60)
    if row["RelativeDate"] <= early_league:
        return "Early"
    elif row["RelativeDate"] <= mid_league:
        return "Mid"
    return "End"

In [96]:
breach_edited["League Lifespan"] = breach_edited.apply(league_lifespan, axis = 1)
breach_edited.head()

,League,Id,Type,Name,BaseType,Value,Confidence,RelativeDate,League Lifespan
71448,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,146.27273,Low,0 days,Early
71449,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,177.88472,Low,1 days,Early
71450,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,232.23558,Low,2 days,Early
71451,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,235.07981,Medium,3 days,Early
71452,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,313.95873,Medium,4 days,Early


# Wrangling Currency Files

In [97]:
currency_to_drop = ["Portal Scroll", "Scroll of Wisdom", "Armourer's Scrap", "Perandus Coin", "Orb of Transmutation", "Blacksmith's Whetstone", "Orb of Augmentation", "Orb of Alteration", "Splinter of Tul", "Chromatic Orb", "Splinter of Esh", "Splinter of Xoph", "Orb of Chance", "Glassblower's Bauble", "Splinter of Uul-Netol", "Silver Coin"]
currency_copy = breach_currency
currency_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9932 entries, 0 to 9931
Data columns (total 6 columns):
League        9932 non-null object
Date          9932 non-null object
Get           9932 non-null object
Pay           9932 non-null object
Value         9932 non-null float64
Confidence    9932 non-null object
dtypes: float64(1), object(5)
memory usage: 465.6+ KB


In [98]:
def currency_info_remove(df):
    currency_to_drop = ["Portal Scroll", "Scroll of Wisdom", "Armourer's Scrap", "Perandus Coin", "Orb of Transmutation", "Blacksmith's Whetstone", "Orb of Augmentation", "Orb of Alteration", "Splinter of Tul", "Chromatic Orb", "Splinter of Esh", "Splinter of Xoph", "Orb of Chance", "Glassblower's Bauble", "Splinter of Uul-Netol", "Silver Coin"]
    df_currency_dropped = df.loc[~(df["Get"].isin(currency_to_drop) | df["Pay"].isin(currency_to_drop))]
    return df_currency_dropped

In [99]:
currency_dropped = currency_info_remove(currency_copy)

In [100]:
currency_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7149 entries, 356 to 9931
Data columns (total 6 columns):
League        7149 non-null object
Date          7149 non-null object
Get           7149 non-null object
Pay           7149 non-null object
Value         7149 non-null float64
Confidence    7149 non-null object
dtypes: float64(1), object(5)
memory usage: 391.0+ KB


In [101]:
currency_cleaned = currency_file_clean(breach_currency)
currency_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7149 entries, 356 to 9931
Data columns (total 7 columns):
League             7149 non-null object
Get                7149 non-null object
Pay                7149 non-null object
Value              7149 non-null float64
Confidence         7149 non-null object
RelativeDate       7149 non-null timedelta64[ns]
League Lifespan    7149 non-null object
dtypes: float64(1), object(5), timedelta64[ns](1)
memory usage: 446.8+ KB


# Generalized File Import and Concatenate

## File Import

In [102]:
item_df_list = []

In [103]:
for filename in glob.glob("*_items.csv"):
    df = pd.read_csv(filename, delimiter = ";", low_memory = False)
    df_edited = item_file_clean(df)
    item_df_list.append(df_edited)

In [104]:
item_df_list[1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4057 entries, 45418 to 246383
Data columns (total 9 columns):
League             4057 non-null object
Id                 4057 non-null int64
Type               4057 non-null object
Name               4057 non-null object
BaseType           4057 non-null object
Value              4057 non-null float64
Confidence         4057 non-null object
RelativeDate       4057 non-null timedelta64[ns]
League Lifespan    4057 non-null object
dtypes: float64(1), int64(1), object(6), timedelta64[ns](1)
memory usage: 317.0+ KB


In [105]:
currency_df_list = []

In [106]:
for filename in glob.glob("*_currency.csv"):
    df = pd.read_csv(filename, delimiter = ";", low_memory = False)
    df_edited = currency_file_clean(df)
    currency_df_list.append(df_edited)

In [107]:
currency_df_list[2].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12387 entries, 369 to 16162
Data columns (total 7 columns):
League             12387 non-null object
Get                12387 non-null object
Pay                12387 non-null object
Value              12387 non-null float64
Confidence         12387 non-null object
RelativeDate       12387 non-null timedelta64[ns]
League Lifespan    12387 non-null object
dtypes: float64(1), object(5), timedelta64[ns](1)
memory usage: 774.2+ KB


## Concatenation

In [108]:
item_df = pd.concat(item_df_list, axis = 0).reset_index(drop = True)
del item_df_list

In [109]:
item_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7741 entries, 0 to 7740
Data columns (total 9 columns):
League             7741 non-null object
Id                 7741 non-null int64
Type               7741 non-null object
Name               7741 non-null object
BaseType           7741 non-null object
Value              7741 non-null float64
Confidence         7741 non-null object
RelativeDate       7741 non-null timedelta64[ns]
League Lifespan    7741 non-null object
dtypes: float64(1), int64(1), object(6), timedelta64[ns](1)
memory usage: 544.4+ KB


In [110]:
item_df.head()

,League,Id,Type,Name,BaseType,Value,Confidence,RelativeDate,League Lifespan
0,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,146.27273,Low,0 days,Early
1,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,177.88472,Low,1 days,Early
2,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,232.23558,Low,2 days,Early
3,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,235.07981,Medium,3 days,Early
4,Breach,1981,UniqueArmour,Tukohama's Fortress,Ebony Tower Shield,313.95873,Medium,4 days,Early


In [111]:
currency_df = pd.concat(currency_df_list, axis = 0).reset_index(drop = True)
del currency_df_list

In [112]:
currency_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38885 entries, 0 to 38884
Data columns (total 7 columns):
League             38885 non-null object
Get                38885 non-null object
Pay                38885 non-null object
Value              38885 non-null float64
Confidence         38885 non-null object
RelativeDate       38885 non-null timedelta64[ns]
League Lifespan    38885 non-null object
dtypes: float64(1), object(5), timedelta64[ns](1)
memory usage: 2.1+ MB


In [113]:
currency_df.head()

,League,Get,Pay,Value,Confidence,RelativeDate,League Lifespan
0,Breach,Orb of Alchemy,Chaos Orb,0.68571,Medium,0 days,Early
1,Breach,Orb of Alchemy,Chaos Orb,0.23652,High,1 days,Early
2,Breach,Orb of Alchemy,Chaos Orb,0.16667,High,2 days,Early
3,Breach,Orb of Alchemy,Chaos Orb,0.19363,High,3 days,Early
4,Breach,Orb of Alchemy,Chaos Orb,0.21778,High,4 days,Early
